<a href="https://colab.research.google.com/github/jmlDC/Truth-Thesis22-23/blob/Scraping-trial/ScrapingNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install requests-html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.4 MB/s 
     |████████████████████████████████| 140 kB 42.3 MB/s 
     |████████████████████████████████| 112 kB 63.4 MB/s 
     |████████████████████████████████| 127 kB 65.9 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=9d8b5d966e9042fe27d8ae8804e0359501a02d0df856b6cf2a513032f3ce99bc
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=e1ba7c3ac96454313ff196f613e25e16ee493dedc31f1e246f1e21cd3bf9f761
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built fake-useragent parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [3]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

from requests_html import HTMLSession 



In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
dir  = "/content/gdrive/MyDrive/THESIS-MS/Truth-Thesis22-23/"


In [6]:
## Politifact 
pagesToGet= 1
upperframe=[]  
site = "politifact"


for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://www.politifact.com/factchecks/list/?page='+str(page)
    print(url)
    
    #an exception might be thrown, so the code should be in a try-except block
    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        page=requests.get(url)                             # this might throw an exception if something goes wrong.
    
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.

    time.sleep(2)   
    soup=BeautifulSoup(page.text,'html.parser')
    frame=[]
    links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    print(len(links))

    
    filename= f"{dir + site}NEWS.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="Statement,Link,Date, Source, Label\n"
    f.write(headers)
    
    for j in links:
        Statement = j.find("div",attrs={'class':'m-statement__quote'}).text.strip()
        Link = "https://www.politifact.com"
        Link += j.find("div",attrs={'class':'m-statement__quote'}).find('a')['href'].strip()
        Date = j.find('div',attrs={'class':'m-statement__body'}).find('footer').text[-14:-1].strip()
        Source = j.find('div', attrs={'class':'m-statement__meta'}).find('a').text.strip()
        Label = j.find('div', attrs ={'class':'m-statement__content'}).find('img',attrs={'class':'c-image__original'}).get('alt').strip()
        frame.append((Statement,Link,Date,Source,Label))
        f.write(Statement.replace(",","^")+","+Link+","+Date.replace(",","^")+","+Source.replace(",","^")+","+Label.replace(",","^")+"\n")
    upperframe.extend(frame)
f.close()
data=pd.DataFrame(upperframe, columns=['Statement','Link','Date','Source','Label'])
data.head()

processing page : 1
https://www.politifact.com/factchecks/list/?page=1
30


,Statement,Link,Date,Source,Label
0,Video shows there weren’t Marines present duri...,https://www.politifact.com/factchecks/2022/sep...,"ember 8, 2022",Viral image,false
1,“The Russian army is withdrawing from Ukraine!...,https://www.politifact.com/factchecks/2022/sep...,"ember 8, 2022",Facebook posts,false
2,"“Here in Nevada, the DMV registers everybody (...",https://www.politifact.com/factchecks/2022/sep...,"ember 8, 2022",Jim Marchant,false
3,Executive Order 14067 will allow the federal g...,https://www.politifact.com/factchecks/2022/sep...,"ember 8, 2022",TikTok posts,false
4,Queen Elizabeth died because of the COVID-19 v...,https://www.politifact.com/factchecks/2022/sep...,"ember 8, 2022",Tweets,false


In [7]:
def extract_source(url):
    agent = {"User-Agent":"Chrome/105.0.0.0"}
    try:
      #use the browser to get the url. This is suspicious command that might blow up.
      # page=requests.get(url)                             # this might throw an exception if something goes wrong.
      source=requests.get(url, headers=agent)
      

    except Exception as e:                                   # this describes what to do if an exception is thrown
      error_type, error_obj, error_info = sys.exc_info()      # get the exception information
      print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
      print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
    
    return source                                               #ignore this page. Abandon this and go back.

    

# def extract_data(source):
#      soup=BeautifulSoup(source, 'lxml')
#      names=soup.findAll('title')
#      for i in names:
#        print(i)  

In [8]:
# extract_data(extract_source(url).text)


In [15]:
## Manila bulletin
pagesToGet= 1
upperframe=[]  
topic = "ukraine war"
site = "manilaBulletin"


for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://mb.com.ph/page/'+str(page)+'/?s='+topic.replace(" ", "+")
    print(url)

    time.sleep(2)   
    # soup=BeautifulSoup(page.text,'html.parser')
    soup=BeautifulSoup(extract_source(url).text, 'html.parser')
    frame=[]
    links=soup.find("article", {'class' : 'content-bar col-12 col-lg-12 order-1 col-xl order-xl-2'}).find_all('li',attrs={'class':'article'})
    print(len(links))
    
    filename= f"{dir + site}NEWS.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="Statement,Link,Date\n"
    f.write(headers)
    
    for j in links:
        try:
          Statement = j.find('h4').find('a').text
          Link = j.find('h4',attrs={'class':'title'}).find('a')['href'].strip()
          Date = j.find('time', {'class': 'time-ago'})['data-time'][0:10].strip()
          frame.append((Statement,Link,Date))
        except Exception as e:
            pass
        f.write(Statement.replace(",","^")+","+Link+","+Date.replace(",","^")+"\n")

    upperframe.extend(frame)
f.close()
data=pd.DataFrame(upperframe, columns=['Statement','Link','Date'])
data.head()

processing page : 1
https://mb.com.ph/page/1/?s=ukraine+war
16


,Statement,Link,Date
0,Gatchalian seeks impact assessment of Russia-U...,https://mb.com.ph/2022/08/04/gatchalian-seeks-...,2022-08-04
1,"US sanctions Putin ‘girlfriend,’ more oligarch...",https://mb.com.ph/2022/08/03/us-sanctions-puti...,2022-08-03
2,Pacquiao-Yoo exhibition match to serve as char...,https://mb.com.ph/2022/07/20/pacquiao-yoo-exhi...,2022-07-20
3,Ukraine war to dominate G20 finance chief meet...,https://mb.com.ph/2022/07/15/ukraine-war-to-do...,2022-07-15
4,Ukraine war could boost illegal drug productio...,https://mb.com.ph/2022/06/27/ukraine-war-could...,2022-06-27


In [14]:
print(links)

[<li class="article">
<div class="article-inner row flex-row-reverse flex-md-row">
<figure class="article-figure col col-sm-auto">
<a class="article-img" href="https://mb.com.ph/2022/08/04/gatchalian-seeks-impact-assessment-of-russia-ukraine-war-on-ph/">
<img alt="viber_image_2022-08-01_19-00-45-992" src="https://mb.com.ph/wp-content/uploads/2022/08/94695.jpeg"/>
</a>
</figure>
<div class="article-info col">
<div class="cat">
<a href="https://mb.com.ph/category/news/national/">National</a> </div>
<h4 class="title"><a href="https://mb.com.ph/2022/08/04/gatchalian-seeks-impact-assessment-of-russia-ukraine-war-on-ph/">Gatchalian seeks impact assessment of Russia-Ukraine war on PH</a></h4>
<div class="desc">
<p>Senator Sherwin Gatchalian ​wants an assessment carried out on the impact of the Russia-Ukraine conflict on Philippine energy security and affordability.</p> </div>
<div class="post-meta">
<span class="time-posted">
<i class="mb-icon-time"></i>
<span class="count"><time class="time-

gdrive	sample_data
